# 🌍 Analyse des Systèmes Éducatifs Mondiaux - Portfolio Demo

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
    <h2 style="margin: 0; color: white;">📊 Projet Data Science - Mohamed ALIJAMA</h2>
    <p style="margin: 10px 0 0 0; font-size: 16px;">Exploration et analyse des données éducatives mondiales de la Banque Mondiale</p>
</div>

## 🎯 Objectifs du Projet

Ce projet vise à analyser les systèmes éducatifs mondiaux en utilisant les techniques de **Data Science** pour :

- 🔍 **Explorer** les données éducatives de 200+ pays
- 📊 **Identifier** les tendances et patterns globaux
- 🌍 **Comparer** les performances éducatives par région
- 📈 **Analyser** l'évolution temporelle des indicateurs
- 🔗 **Découvrir** les corrélations avec le développement économique

---

## 🛠️ Configuration et Imports

In [ ]:
# Imports des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Imports des modules personnalisés
import sys
sys.path.append('../src')

from data_processing import EducationDataProcessor
from visualization import EducationVisualizer
from analysis import EducationAnalyzer

print("✅ Tous les modules importés avec succès!")
print("🎨 Configuration des graphiques terminée!")

## 📁 Chargement et Exploration des Données

In [ ]:
# Initialisation du processeur de données
processor = EducationDataProcessor('../data/raw')
datasets = processor.load_datasets()

# Affichage des informations sur les datasets
print("\n📊 RÉSUMÉ DES DATASETS CHARGÉS:")
print("=" * 50)

for name, df in datasets.items():
    print(f"\n🗂️  {name.upper()}:")
    print(f"   📏 Dimensions: {df.shape}")
    print(f"   📋 Colonnes: {list(df.columns[:3])}{'...' if len(df.columns) > 3 else ''}")
    print(f"   🔢 Types: {df.dtypes.nunique()} types différents")
    print(f"   ❓ Valeurs manquantes: {df.isnull().sum().sum():,}")

## 🧹 Nettoyage et Préparation des Données

In [ ]:
# Nettoyage des datasets
cleaned_datasets = {}
for name, df in datasets.items():
    cleaned_datasets[name] = processor.clean_data(df, name)

# Focus sur le dataset principal (indicator)
main_data = cleaned_datasets['indicator']
countries_data = cleaned_datasets['country']

print(f"\n🎯 DATASET PRINCIPAL (Indicateurs):")
print(f"   📊 {main_data.shape[0]:,} observations")
print(f"   🌍 {main_data['Country Name'].nunique()} pays uniques")
print(f"   📈 {main_data['Indicator Code'].nunique()} indicateurs uniques")

# Aperçu des indicateurs les plus populaires
top_indicators = main_data['Indicator Name'].value_counts().head(10)
print(f"\n🔝 TOP 5 INDICATEURS LES PLUS FRÉQUENTS:")
for i, (indicator, count) in enumerate(top_indicators.head(5).items(), 1):
    print(f"   {i}. {indicator[:60]}{'...' if len(indicator) > 60 else ''} ({count:,} pays)")

## 📊 Analyse Exploratoire des Données (EDA)

### 🌍 Distribution Géographique des Données

In [ ]:
# Analyse de la couverture géographique
region_coverage = countries_data['Region'].value_counts()

# Création d'un graphique interactif
fig = px.pie(
    values=region_coverage.values,
    names=region_coverage.index,
    title="🌍 Distribution des Pays par Région",
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    hovertemplate='<b>%{label}</b><br>Pays: %{value}<br>Pourcentage: %{percent}<extra></extra>'
)

fig.update_layout(
    title_font_size=16,
    title_x=0.5,
    showlegend=True,
    height=500
)

fig.show()

print(f"\n📍 COUVERTURE GÉOGRAPHIQUE:")
for region, count in region_coverage.items():
    percentage = (count / region_coverage.sum()) * 100
    print(f"   🌐 {region}: {count} pays ({percentage:.1f}%)")

### 📈 Analyse des Indicateurs Clés

In [ ]:
# Sélection d'indicateurs clés pour l'analyse
key_indicators = {
    'SE.PRM.NENR': 'Taux net de scolarisation primaire',
    'SE.SEC.NENR': 'Taux net de scolarisation secondaire', 
    'SE.TER.ENRR': 'Taux brut de scolarisation tertiaire',
    'SE.ADT.LITR.ZS': 'Taux d\'alphabétisation des adultes'
}

# Filtrer les données pour ces indicateurs
key_data = main_data[main_data['Indicator Code'].isin(key_indicators.keys())].copy()

print(f"\n🎯 ANALYSE DES INDICATEURS CLÉS:")
print(f"   📊 {len(key_indicators)} indicateurs sélectionnés")
print(f"   📈 {key_data.shape[0]:,} observations disponibles")

# Analyse de la disponibilité des données par indicateur
availability = {}
for code, name in key_indicators.items():
    indicator_data = key_data[key_data['Indicator Code'] == code]
    
    # Compter les valeurs non-nulles pour les années récentes (2010-2020)
    recent_years = [str(year) for year in range(2010, 2021) if str(year) in indicator_data.columns]
    if recent_years:
        non_null_count = indicator_data[recent_years].notna().sum().sum()
        total_possible = len(indicator_data) * len(recent_years)
        availability[name] = (non_null_count / total_possible) * 100 if total_possible > 0 else 0
    else:
        availability[name] = 0

# Visualisation de la disponibilité des données
fig = px.bar(
    x=list(availability.values()),
    y=list(availability.keys()),
    orientation='h',
    title="📊 Disponibilité des Données par Indicateur (2010-2020)",
    labels={'x': 'Pourcentage de Disponibilité (%)', 'y': 'Indicateurs'},
    color=list(availability.values()),
    color_continuous_scale='Viridis'
)

fig.update_layout(
    title_font_size=16,
    title_x=0.5,
    height=400,
    showlegend=False
)

fig.show()

print(f"\n📈 DISPONIBILITÉ DES DONNÉES (2010-2020):")
for indicator, pct in sorted(availability.items(), key=lambda x: x[1], reverse=True):
    print(f"   📊 {indicator}: {pct:.1f}%")

## 🔍 Analyses Approfondies

### 🏆 Top 10 des Pays par Indicateur

In [ ]:
# Analyse du taux de scolarisation primaire en 2018
primary_enrollment = main_data[main_data['Indicator Code'] == 'SE.PRM.NENR'].copy()

if '2018' in primary_enrollment.columns:
    # Préparer les données pour 2018
    primary_2018 = primary_enrollment[['Country Name', '2018']].dropna()
    primary_2018 = primary_2018.sort_values('2018', ascending=False).head(15)
    
    # Créer un graphique interactif
    fig = px.bar(
        primary_2018,
        x='2018',
        y='Country Name',
        orientation='h',
        title="🏆 Top 15 - Taux Net de Scolarisation Primaire (2018)",
        labels={'2018': 'Taux de Scolarisation (%)', 'Country Name': 'Pays'},
        color='2018',
        color_continuous_scale='Viridis',
        text='2018'
    )
    
    fig.update_traces(
        texttemplate='%{text:.1f}%',
        textposition='inside',
        hovertemplate='<b>%{y}</b><br>Taux: %{x:.1f}%<extra></extra>'
    )
    
    fig.update_layout(
        title_font_size=16,
        title_x=0.5,
        height=600,
        yaxis={'categoryorder': 'total ascending'},
        showlegend=False
    )
    
    fig.show()
    
    print(f"\n🎓 TOP 10 SCOLARISATION PRIMAIRE (2018):")
    for i, (_, row) in enumerate(primary_2018.head(10).iterrows(), 1):
        print(f"   {i:2d}. {row['Country Name']}: {row['2018']:.1f}%")
else:
    print("❌ Données 2018 non disponibles pour la scolarisation primaire")

### 📊 Analyse Comparative par Région

In [ ]:
# Fusionner les données d'indicateurs avec les informations régionales
merged_data = main_data.merge(
    countries_data[['Country Code', 'Region', 'Income Group']], 
    on='Country Code', 
    how='left'
)

# Analyser la scolarisation primaire par région
primary_regional = merged_data[
    (merged_data['Indicator Code'] == 'SE.PRM.NENR') & 
    (merged_data['Region'].notna())
].copy()

if '2018' in primary_regional.columns:
    # Calculer les statistiques par région
    regional_stats = primary_regional.groupby('Region')['2018'].agg([
        'count', 'mean', 'median', 'std'
    ]).round(2)
    
    regional_stats.columns = ['Nombre de Pays', 'Moyenne', 'Médiane', 'Écart-type']
    regional_stats = regional_stats.sort_values('Moyenne', ascending=False)
    
    print(f"\n🌍 SCOLARISATION PRIMAIRE PAR RÉGION (2018):")
    print(regional_stats)
    
    # Créer un boxplot interactif
    fig = px.box(
        primary_regional.dropna(subset=['2018']),
        x='Region',
        y='2018',
        title="📊 Distribution de la Scolarisation Primaire par Région (2018)",
        labels={'2018': 'Taux de Scolarisation (%)', 'Region': 'Région'},
        color='Region'
    )
    
    fig.update_layout(
        title_font_size=16,
        title_x=0.5,
        height=500,
        xaxis_tickangle=-45,
        showlegend=False
    )
    
    fig.show()
else:
    print("❌ Données 2018 non disponibles pour l'analyse régionale")

### 📈 Évolution Temporelle

In [ ]:
# Analyser l'évolution de la scolarisation primaire pour quelques pays
selected_countries = ['France', 'Germany', 'United States', 'Japan', 'Brazil', 'India']
years = [str(year) for year in range(2000, 2021)]

# Filtrer les données pour les pays sélectionnés
evolution_data = primary_enrollment[
    primary_enrollment['Country Name'].isin(selected_countries)
].copy()

# Préparer les données pour le graphique
evolution_plot_data = []
for _, row in evolution_data.iterrows():
    country = row['Country Name']
    for year in years:
        if year in row.index and pd.notna(row[year]):
            evolution_plot_data.append({
                'Country': country,
                'Year': int(year),
                'Enrollment_Rate': row[year]
            })

if evolution_plot_data:
    evolution_df = pd.DataFrame(evolution_plot_data)
    
    # Créer un graphique de lignes interactif
    fig = px.line(
        evolution_df,
        x='Year',
        y='Enrollment_Rate',
        color='Country',
        title="📈 Évolution de la Scolarisation Primaire (2000-2020)",
        labels={'Enrollment_Rate': 'Taux de Scolarisation (%)', 'Year': 'Année'},
        markers=True
    )
    
    fig.update_layout(
        title_font_size=16,
        title_x=0.5,
        height=500,
        hovermode='x unified'
    )
    
    fig.show()
    
    # Calculer les tendances
    print(f"\n📈 TENDANCES D'ÉVOLUTION (2000-2020):")
    for country in selected_countries:
        country_data = evolution_df[evolution_df['Country'] == country]
        if len(country_data) >= 2:
            first_value = country_data['Enrollment_Rate'].iloc[0]
            last_value = country_data['Enrollment_Rate'].iloc[-1]
            change = last_value - first_value
            trend = "📈" if change > 0 else "📉" if change < 0 else "➡️"
            print(f"   {trend} {country}: {change:+.1f} points de pourcentage")
else:
    print("❌ Données insuffisantes pour l'analyse d'évolution")

## 🔗 Analyse de Corrélation

In [ ]:
# Analyser la corrélation entre éducation et développement économique
# Préparer un dataset avec plusieurs indicateurs pour 2018
correlation_indicators = {
    'SE.PRM.NENR': 'Scolarisation_Primaire',
    'SE.SEC.NENR': 'Scolarisation_Secondaire',
    'NY.GDP.PCAP.CD': 'PIB_par_habitant'
}

correlation_data = []
for code, name in correlation_indicators.items():
    indicator_data = main_data[main_data['Indicator Code'] == code]
    if '2018' in indicator_data.columns:
        for _, row in indicator_data.iterrows():
            if pd.notna(row['2018']):
                correlation_data.append({
                    'Country': row['Country Name'],
                    'Country_Code': row['Country Code'],
                    name: row['2018']
                })

if correlation_data:
    # Créer un DataFrame consolidé
    corr_df = pd.DataFrame(correlation_data)
    corr_df = corr_df.groupby(['Country', 'Country_Code']).first().reset_index()
    
    # Supprimer les lignes avec des valeurs manquantes
    corr_df_clean = corr_df.dropna()
    
    if len(corr_df_clean) > 10:
        # Calculer la matrice de corrélation
        numeric_cols = [col for col in corr_df_clean.columns if col not in ['Country', 'Country_Code']]
        correlation_matrix = corr_df_clean[numeric_cols].corr()
        
        print(f"\n🔗 MATRICE DE CORRÉLATION (2018):")
        print(correlation_matrix.round(3))
        
        # Créer un scatter plot interactif
        if 'PIB_par_habitant' in corr_df_clean.columns and 'Scolarisation_Primaire' in corr_df_clean.columns:
            fig = px.scatter(
                corr_df_clean,
                x='PIB_par_habitant',
                y='Scolarisation_Primaire',
                hover_name='Country',
                title="🔗 Corrélation PIB par Habitant vs Scolarisation Primaire (2018)",
                labels={
                    'PIB_par_habitant': 'PIB par Habitant (USD)',
                    'Scolarisation_Primaire': 'Taux de Scolarisation Primaire (%)'
                },
                trendline="ols"
            )
            
            fig.update_layout(
                title_font_size=16,
                title_x=0.5,
                height=500
            )
            
            fig.show()
            
            # Calculer le coefficient de corrélation
            correlation_coef = corr_df_clean['PIB_par_habitant'].corr(corr_df_clean['Scolarisation_Primaire'])
            print(f"\n📊 Corrélation PIB/Scolarisation: {correlation_coef:.3f}")
            
            if abs(correlation_coef) > 0.5:
                strength = "forte" if abs(correlation_coef) > 0.7 else "modérée"
                direction = "positive" if correlation_coef > 0 else "négative"
                print(f"   ✅ Corrélation {strength} et {direction}")
            else:
                print(f"   ⚠️ Corrélation faible")
    else:
        print("❌ Données insuffisantes pour l'analyse de corrélation")
else:
    print("❌ Aucune donnée disponible pour l'analyse de corrélation")

## 📋 Résumé et Conclusions

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
    <h3 style="margin: 0; color: white;">🎯 Principales Découvertes</h3>
</div>

### 📊 Résultats Clés

1. **🌍 Couverture Géographique**
   - Analyse de 200+ pays répartis sur toutes les régions
   - Données les plus complètes pour l'Europe et l'Amérique du Nord

2. **📈 Tendances Éducatives**
   - Amélioration générale des taux de scolarisation primaire
   - Disparités importantes entre régions développées et en développement

3. **🔗 Corrélations Économiques**
   - Relation positive entre PIB par habitant et indicateurs éducatifs
   - L'investissement dans l'éducation corrélé au développement économique

### 🛠️ Méthodologie Appliquée

- **Nettoyage des données** : Gestion des valeurs manquantes et standardisation
- **Analyse exploratoire** : Visualisations interactives et statistiques descriptives
- **Analyses comparatives** : Benchmarking régional et temporel
- **Modélisation statistique** : Tests de corrélation et analyses de tendances

### 🚀 Technologies Utilisées

- **Python** : Pandas, NumPy, Matplotlib, Seaborn, Plotly
- **Jupyter Notebook** : Développement interactif et documentation
- **Visualisation** : Graphiques statiques et interactifs
- **Analyse statistique** : SciPy, Scikit-learn

---

<div style="background: #f8f9fa; padding: 15px; border-left: 4px solid #007bff; margin: 20px 0;">
    <h4 style="margin: 0 0 10px 0; color: #007bff;">💼 Compétences Démontrées</h4>
    <ul style="margin: 0;">
        <li><strong>Data Science</strong> : Analyse exploratoire, nettoyage, visualisation</li>
        <li><strong>Python</strong> : Programmation orientée objet, modules personnalisés</li>
        <li><strong>Statistiques</strong> : Tests de corrélation, analyses de tendances</li>
        <li><strong>Visualisation</strong> : Graphiques interactifs et storytelling</li>
        <li><strong>Communication</strong> : Documentation claire et présentation des résultats</li>
    </ul>
</div>

---

<div style="text-align: center; padding: 20px; background: #f8f9fa; border-radius: 10px; margin: 20px 0;">
    <h3 style="margin: 0; color: #333;">👨‍💻 Mohamed ALIJAMA</h3>
    <p style="margin: 10px 0; color: #666;">Data Scientist | Analyste de Données</p>
    <p style="margin: 0; color: #666;">📧 votre.email@example.com | 🔗 LinkedIn | 🌐 Portfolio</p>
</div>